In [22]:
import os
import warnings
warnings.filterwarnings('ignore')
import skimage
from skimage.data import imread as skimg_imread
from skimage import transform 
from skimage.color import rgb2gray
import numpy

def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) 
                      if f.endswith(".ppm")]
        for f in file_names:
            images.append(skimg_imread(f))
            labels.append(int(d))
    return images, labels

data_folder = r'/Users/lantian/Desktop/Ivan/Marketing perso/Personal-Training/Tensorflow/data/Datacamp tutorial'

train_data_directory = os.path.join(data_folder, "Training")
test_data_directory = os.path.join(data_folder, "Testing")
images, labels = load_data(train_data_directory)
images28 = [transform.resize(image, (28, 28)) for image in images]
images28 = numpy.array(images28)
images28_gray = rgb2gray(images28)

#### Fully connected layer with a hidden layer

In [23]:
import tensorflow as tf
# Initialize placeholders 
x = tf.placeholder(dtype = tf.float32, shape = [None, 28, 28])
y = tf.placeholder(dtype = tf.int32, shape = [None])

In [24]:
images_flat = tf.contrib.layers.flatten(x)
n_hidden_rule_thumb = int((2/3)*28*28 + 62)

Rule of thumb for number of neuron in hidden layer <a href="https://www.ijcsmc.com/docs/papers/November2014/V3I11201499a19.pdf">here</a><hr>

<i>
<li>The number of hidden neurons should be in the range between the size of the input layer and the size of the output layer</li>

<li>The number of hidden neurons should be 2/3 of the input layer size, plus the size of theoutput layer</li>

<li>The number of hidden neurons should be less than twice the input layer size</li>
</i>

In [25]:
n_hidden_rule_thumb

584

In [26]:
hidden = tf.contrib.layers.fully_connected(images_flat, n_hidden_rule_thumb, tf.nn.relu)
logits = tf.contrib.layers.fully_connected(hidden, 62, tf.nn.relu)

In [27]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
loss = tf.reduce_mean(cross_entropy)
train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

prediction = tf.argmax(logits, 1)
accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))

#### TRAIN

In [28]:
import pandas

In [29]:
s_loss = pandas.Series(index=range(201))

tf.set_random_seed(1234)
sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(201):
    _, loss_value, accuracy_val = sess.run([train_op, loss, accuracy], feed_dict={x: images28_gray, y: labels})
    s_loss.loc[i] = loss_value
    if i % 10 == 0:
        # print('EPOCH', i)
        # print("Accuracy: ", accuracy_val)
        print("Loss: ", loss_value)
        pass

# sess.close()

Loss:  4.17961
Loss:  3.2956
Loss:  2.88491
Loss:  2.62471
Loss:  2.45488
Loss:  2.34085
Loss:  2.25745
Loss:  2.19303
Loss:  2.14196
Loss:  2.10004
Loss:  2.06467
Loss:  2.03437
Loss:  2.0082
Loss:  1.98547
Loss:  1.96568
Loss:  1.94886
Loss:  1.93356
Loss:  1.92064
Loss:  1.90906
Loss:  1.89862
Loss:  1.88992


#### TEST

In [30]:
# Import `skimage`
from skimage import transform

# Load the test data
test_images, test_labels = load_data(test_data_directory)

# Transform the images to 28 by 28 pixels
test_images28 = [transform.resize(image, (28, 28)) for image in test_images]

# Convert to grayscale
from skimage.color import rgb2gray
test_images28_gray = rgb2gray(numpy.array(test_images28))

# Run predictions against the full test set.
predicted = sess.run(prediction, feed_dict={x: test_images28_gray})

# Calculate correct matches 
match_count = sum([int(y == y_) for y, y_ in zip(test_labels, predicted)])

# Calculate the accuracy
accuracy = match_count / len(test_labels)

# Print the accuracy
print("Accuracy: {:.3f}".format(accuracy))

Accuracy: 0.563


Only a disappointing improvement from 54.4% previously...

https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-ii-hyper-parameter-42efca01e5d7